In [75]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
import datetime as dt

In [76]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:
tt = ['ADBE', 'AMT', 'PLD', 'VICI', 'ATVI', 'CVS', 'BIO', 'JNJ'
      , 'NVDA', 'IBM', 'NFLX', 'GOOG', 'AAPL'
      , 'AMZN', 'EBAY', 'SBUX', 'TSLA']
# tt = ['NDAQ', 'SCHW', 'JPM']
sc = 'Voting_EF_'
# tt = ['AAPL', 'MSFT', 'TSLA']
trend1 = 'uptrend'
# trend1 = 'downtrend'

In [106]:
# Full_data_path = '/content/drive/MyDrive/merge/'
Full_data_path = '/content/drive/MyDrive/stock_GCN_info/multiProt/'
UD_file_path = '/content/drive/MyDrive/stock_results_tuned/'
# gcn_file_path = '/content/drive/MyDrive/stock_results_tuned/GCN_LSTMulti/'
gcn_file_path = '/content/drive/MyDrive/stock_results_tuned/GCN_LSTMulti/'
data_port_path = '/content/drive/MyDrive/stock_data/'

In [107]:
def fd(trend):
  if trend1 == 'uptrend':
    trend = '_uptrend'
    folder = 'Uptrend/'
  elif trend1 == 'downtrend':
    trend = '_downtrend'
    folder = 'Downtrend/'
  return trend, folder

In [108]:
def sd(trend):
  if trend == '_downtrend':
    start_date = '2022-01-03' # It will be later once file is merged with stacking model preds
    end_date = '2022-09-20'
  elif trend == '_uptrend':
    start_date = '2021-07-01'
    end_date = '2021-12-31'
  return start_date, end_date

In [109]:
trend = fd(trend1)[0]
folder = fd(trend1)[1]

In [110]:
start_date = sd(trend)[0]
end_date = sd(trend)[1]
print(start_date)
print(end_date)

2021-07-01
2021-12-31


# Portfolio analysis function

In [111]:
# def buy_stock(real_movement, signal, initial_money=10000, max_buy=10, max_sell=10, force_buy=True):
#     """
#     initial_money = 10000, ignore what kind of currency
#     delay = how much interval you want to delay to change our decision from buy to sell, vice versa
#     initial_state = 1 is buy, 0 is sell
#     real_movement = actual movement in the real world
#     max_buy = max quantity for share to buy
#     max_sell = max quantity for share to sell
#     """
#     starting_money = initial_money
#     states_sell = []
#     states_buy = []
#     current_inventory = 0

#     def buy(i, initial_money, current_inventory):
#         shares = initial_money // real_movement[i]
#         print("Shares: " + str(shares))
#         if shares < 1:
#             print(
#                 'day %d: total balances %f, not enough money to buy a unit price %f'
#                 % (i, initial_money, real_movement[i])
#             )
#         else:
#             if shares > max_buy:
#                 buy_units = max_buy
#             else:
#                 buy_units = shares
#             initial_money -= buy_units * real_movement[i]
#             current_inventory += buy_units
#             print(
#                 'day %d: buy %d units at price %f, total balance %f'
#                 % (i, buy_units, buy_units * real_movement[i], initial_money)
#             )
#             states_buy.append(0)
#         return initial_money, current_inventory

#     #for i in range(real_movement.shape[0] - int(0.025 * len(df))):
#     for i in range(real_movement.shape[0]-1):
#         state = signal[i]
#         print("Prediction: " + str(state))

#         if force_buy and i==0:
#             state = 1 # Ignoring a possibly negative prediction and force buy stocks on first day

#         if state == 1:
#             initial_money, current_inventory = buy(
#                 i, initial_money, current_inventory
#             )
#             states_buy.append(i)
#         elif state == 0:
#             if current_inventory == 0:
#                     print('day %d: cannot sell anything, inventory 0' % (i))
#             else:
#                 if current_inventory > max_sell:
#                     sell_units = max_sell
#                 else:
#                     sell_units = current_inventory
#                 current_inventory -= sell_units
#                 total_sell = sell_units * real_movement[i]
#                 initial_money += total_sell
#                 try:
#                     invest = (
#                         (real_movement[i] - real_movement[states_buy[-1]])
#                         / real_movement[states_buy[-1]]
#                     ) * 100
#                 except:
#                     invest = 0
#                 print(
#                     'day %d, sell %d units at price %f, investment %f %%, total balance %f,'
#                     % (i, sell_units, total_sell, invest, initial_money)
#                 )
#             states_sell.append(i)

#     # New: Sell remaining stocks to compute final value of portfolio

#     if current_inventory > 0:
#         sell_units = current_inventory
#         current_inventory -= sell_units
#         total_sell = sell_units * real_movement[i]
#         initial_money += total_sell
#         try:
#             invest = (
#                 (real_movement[i] - real_movement[states_buy[-1]])
#                 / real_movement[states_buy[-1]]
#             ) * 100
#         except:
#             invest = 0
#         print(
#             'Final : day %d, sell %d units at price %f, investment %f %%, total balance %f,'
#             % (i, sell_units, total_sell, invest, initial_money)
#         )
#     states_sell.append(i)

#     invest = ((initial_money - starting_money) / starting_money) * 100
#     total_gains = initial_money - starting_money
#     return states_buy, states_sell, total_gains, invest


In [112]:
#FFInial Voting Decision
def buy_stock(real_movement, signal, initial_money=10000, max_buy=10, max_sell=10, force_buy=True, conservative=True, max_days=3):
    starting_money = initial_money
    states_sell = []
    states_buy = []
    current_inventory = 0
    no_sell = 0
    no_buy = 0

    def buy(i, initial_money, current_inventory):
        shares = initial_money // real_movement[i]
        print("Shares: " + str(shares))
        if shares < 1:
            print(
                'day %d: total balance %f, not enough money to buy a unit price %f'
                % (i, initial_money, real_movement[i])
            )
        else:
            if shares > max_buy:
                buy_units = max_buy
            else:
                buy_units = shares
            initial_money -= buy_units * real_movement[i]
            current_inventory += buy_units
            print(
                'day %d: buy %d units at price %f, total balance %f'
                % (i, buy_units, buy_units * real_movement[i], initial_money)
            )
            states_buy.append(i)
        return initial_money, current_inventory

    for i in range(real_movement.shape[0]-1):
        state = signal[i]
        print("Prediction: " + str(state))

        if force_buy and i == 0:
            state = 0  # Ignoring a possibly negative prediction and force buy stocks on the first day

        if state == 0:
            if conservative and no_sell < max_days:
                no_sell += 1
                if no_sell == max_days:
                    if current_inventory > 0:
                        sell_units = current_inventory
                        current_inventory -= sell_units
                        total_sell = sell_units * real_movement[i]
                        initial_money += total_sell
                        try:
                            invest = (
                                (real_movement[i] - real_movement[states_buy[-1]])
                                / real_movement[states_buy[-1]]
                            ) * 100
                        except:
                            invest = 0
                        print(
                            'day %d, sell %d units at price %f, investment %f %%, total balance %f,'
                            % (i, sell_units, total_sell, invest, initial_money)
                        )
                        states_sell.append(i)
            else:
                initial_money, current_inventory = buy(i, initial_money, current_inventory)
                states_buy.append(i)
                no_sell = 0
        elif state == 1:
            if conservative and no_buy < max_days:
                no_buy += 1
                if no_buy == max_days and initial_money >= (max_buy * real_movement[i]):
                    buy_units = max_buy
                    initial_money -= buy_units * real_movement[i]
                    current_inventory += buy_units
                    print(
                        'day %d: buy %d units at price %f, total balance %f'
                        % (i, buy_units, buy_units * real_movement[i], initial_money)
                    )
                    states_buy.append(i)
                    no_buy = 0
            else:
                if current_inventory == 0:
                    print('day %d: cannot sell anything, inventory 0' % (i))
                else:
                    if current_inventory > max_sell:
                        sell_units = max_sell
                    else:
                        sell_units = current_inventory
                    current_inventory -= sell_units
                    total_sell = sell_units * real_movement[i]
                    initial_money += total_sell
                    try:
                        invest = (
                            (real_movement[i] - real_movement[states_buy[-1]])
                            / real_movement[states_buy[-1]]
                        ) * 100
                    except:
                        invest = 0
                    print(
                        'day %d, sell %d units at price %f, investment %f %%, total balance %f,'
                        % (i, sell_units, total_sell, invest, initial_money)
                    )
                    states_sell.append(i)
                    no_buy = 0

    if current_inventory > 0:
        sell_units = current_inventory
        current_inventory -= sell_units
        total_sell = sell_units * real_movement[i]
        initial_money += total_sell
        try:
            invest = (
                (real_movement[i] - real_movement[states_buy[-1]])
                / real_movement[states_buy[-1]]
            ) * 100
        except:
            invest = 0
        print(
            'Final : day %d, sell %d units at price %f, investment %f %%, total balance %f,'
            % (i, sell_units, total_sell, invest, initial_money)
        )
    states_sell.append(i)

    invest = ((initial_money - starting_money) / starting_money) * 100
    total_gains = initial_money - starting_money
    return states_buy, states_sell, total_gains, invest

In [113]:
# importing all the library
# all are not used but i have still kept to avoid errors
import pandas as pd
import math
import numpy as np
import sys
from sklearn import metrics
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.model_selection import train_test_split,cross_val_score,KFold,StratifiedKFold
from sklearn.tree import DecisionTreeClassifier,plot_tree,export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors, metrics,datasets,svm,linear_model, tree, ensemble
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.datasets import make_classification,load_iris,load_digits
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score,ParameterGrid,GridSearchCV,train_test_split
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer,HashingVectorizer, CountVectorizer
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression,Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RandomizedSearchCV,train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,Normalizer,StandardScaler

from scipy.stats.mstats import gmean
from numpy import mean,std
from matplotlib.colors import ListedColormap

from datetime import datetime

import matplotlib.pyplot as plt
import warnings
import time
import seaborn as sns
import random
sns.set()
warnings.filterwarnings("ignore", category=DeprecationWarning)
plt.style.use('fivethirtyeight')

In [114]:
def plot_model_performance(ticker, close, states_buy, states_sell, model_name):
    #close = merged['close']
    fig = plt.figure(figsize = (15,5))
    plt.plot(close, color='r', lw=2.)
    plt.plot(close, '^', markersize=10, color='m', label = 'buying signal (' + model_name + ')', markevery = states_buy)
    plt.plot(close, 'v', markersize=10, color='k', label = 'selling signal (' + model_name + ')', markevery = states_sell)
    #plt.title('Total Gains (USD) %f, Total Gains (Perc.) %f%%'%(total_gains, invest))
    plt.legend(fontsize='large')
    #plt.show()
    # plt.savefig(gcn_file_path + 'clasrport_plot/'+ticker + '_model_' + model_name + '.pdf', format='pdf')
    plt.savefig('/content/drive/MyDrive/stock_GCN_info/multiPort/'+ticker + '_model_' + model_name + '.pdf', format='pdf')
    plt.close(fig)

In [115]:
# Do nothing strategy
def buy_hold_strategy(real_movement, initial_money=10000):
    """
    initial_money = 10000, ignore what kind of currency
    delay = how much interval you want to delay to change our decision from buy to sell, vice versa
    real_movement = actual movement in the real world
    """
    starting_money = initial_money
    current_inventory = 0

    def buy(i, initial_money, current_inventory):
        shares = initial_money // real_movement[i]
        print("Shares: " + str(shares))
        if shares < 1:
            print(
                'day %d: total balances %f, not enough money to buy a unit price %f'
                % (i, initial_money, real_movement[i])
            )
        else:
            buy_units = shares
            initial_money -= buy_units * real_movement[i]
            current_inventory += buy_units
            print(
                'day %d: buy %d units at price %f, total balance %f'
                % (i, buy_units, buy_units * real_movement[i], initial_money)
            )
            states_buy.append(0)
        return initial_money, current_inventory

    i = 0    # First day
    initial_money, current_inventory = buy(i, initial_money, current_inventory)
    states_buy.append(i)

    i = real_movement.shape[0]-1   # Last day
    sell_units = current_inventory
    current_inventory -= sell_units
    total_sell = sell_units * real_movement[i]
    initial_money += total_sell

    try:
        invest = (
            (real_movement[i] - real_movement[states_buy[-1]])
            / real_movement[states_buy[-1]]
        ) * 100
    except:
        invest = 0

    print(
        'Final : day %d, sell %d units at price %f, investment %f %%, total balance %f,'
        % (i, sell_units, total_sell, invest, initial_money)
    )
    states_sell.append(i)

    invest = ((initial_money - starting_money) / starting_money) * 100
    total_gains = initial_money - starting_money

    return states_buy, states_sell, total_gains, invest

#Load data

In [116]:
for i in tt:
  Full_data = pd.read_csv(Full_data_path + i + '.csv')
  Full_data['Unnamed: 0'] = Full_data['Unnamed: 0'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y').strftime('%m/%d/%Y'))

  triker1 = i.lower()

  test = pd.read_csv(gcn_file_path + triker1+ trend +'_GCN_LSTMulti.csv')
  test = test.drop('Unnamed: 0', axis = 1)
  test.columns = ['multimodal Prediction_OLD', 'Ground Truth', 'Unnamed: 0']
  test['Unnamed: 0'] = test['Unnamed: 0'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%m/%d/%Y'))

  merge = pd.merge(Full_data, test, on=['Unnamed: 0', 'Ground Truth'])
  # print(merge)

In [117]:
merge = Full_data
merge

,Unnamed: 0.1,Ground Truth,Unnamed: 0,Bi-LSTM,CNN-Seq2Seq,Attention-CNN-LSTM,Dilated CNN Seq2Seq,CNN-LSTM,GRU-CNN,date,LSTM Predictions,Polarity Predictions,Arima Predictions,Polarity Scores,GBTsPreds,predictionsProbLSTM,multimodal Predictions
0,0,0,07/01/2021,0,0,0,1,1,0,2021-07-01,1,0,1,0.351667,1,9.999552e-01,1
1,1,1,07/02/2021,0,0,0,1,0,0,2021-07-02,0,1,1,0.563618,1,4.393649e-05,1
2,3,0,07/07/2021,0,0,0,0,0,0,2021-07-07,0,1,1,0.419250,1,6.486464e-04,1
3,4,1,07/08/2021,0,0,0,0,0,0,2021-07-08,1,0,1,0.126692,0,9.999910e-01,1
4,5,1,07/09/2021,0,0,0,0,0,0,2021-07-09,1,0,0,0.380552,1,9.999943e-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,175,1,09/14/2022,1,1,1,1,0,0,2022-09-14,1,1,1,0.449560,0,9.999993e-01,1
284,176,1,09/15/2022,1,1,1,1,0,0,2022-09-15,1,0,0,0.313343,1,9.999990e-01,1
285,177,0,09/16/2022,1,1,1,1,0,0,2022-09-16,0,1,0,0.557500,1,1.638767e-07,1
286,178,1,09/19/2022,1,1,1,1,0,0,2022-09-19,0,1,0,0.450824,1,1.196922e-05,1


In [118]:
len(tt)

17

# PA

In [119]:
for ele in tt:
  print(ele)

ADBE
AMT
PLD
VICI
ATVI
CVS
BIO
JNJ
NVDA
IBM
NFLX
GOOG
AAPL
AMZN
EBAY
SBUX
TSLA


In [120]:
# import sys

# # Define the max_days and conservative parameters
# max_days = 3
# conservative = True
# initial_shares = 10

# loggers = {}

# for ticker in tt:
#     loggers[ticker] = []

# for ele in tt:
#     initial = 10000 / len(tt)  # Capital at starting point
#     portfolio_cols = ['LSTM Predictions', 'Polarity Predictions', 'GBTsPreds', 'Arima Predictions', 'multimodal Predictions']
#     models = ['LSTM', 'Polarity', 'GBTs', 'ARIMA', 'multimodal']

#     buy_indicator = 0
#     sell_indicator = 1

#     # 'Ground Truth': incorrect metrics for models
#     max_stocks = [1, 5, 10, 20, 40, 80, 100]
#     logger = []
#     logger.append("Model,Indicators,Force Buy,Max Stocks,Gains USD,Gains Perc")

#     # Load data: stocks and model predictions
#     triker1 = ele.lower()
#     df = pd.read_csv(data_port_path + triker1 + '_Simon/stocks_ts_' + ele + '_2021-1-4_2022-9-20.csv')
#     df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
#     filtered = df.loc[(df['date'] >= start_date)].loc[(df['date'] < end_date)]

#     predictors = merge

#     try:
#         predictors['date'] = pd.to_datetime(predictors['Unnamed: 0'], format='%m/%d/%Y')
#     except:
#         predictors['dates'] = pd.to_datetime(predictors['Unnamed: 0'], format='%m/%d/%y')

#     merged = predictors.merge(filtered, on='date')
#     merged['NextDayTrend'] = merged['NextDayTrend'].replace({True: 1, False: 0})

#     print(len(merged))
#     merged

#     # Simulation
#     for col in portfolio_cols:
#         for force_buy in [True, False]:
#             for num_stocks in max_stocks:
#                 for ticker, logger in loggers.items():
#                     if ele == ticker:
#                         print("\n\n\nTesting model " + col + " with force_buy=" + str(force_buy) + " and max_stocks=" + str(num_stocks) + ":\n")

#                         # Reset the initial state for each simulation scenario
#                         budget = initial
#                         shares = initial_shares

#                         # Call the buy_stock function to perform the simulation
#                         states_buy, states_sell, total_gains, invest = buy_stock(merged['close'], merged[col], initial_money=budget, max_buy=num_stocks, max_sell=num_stocks, force_buy=force_buy, conservative=True, max_days=3)

#                         # Print the results
#                         print("States Buy: ", states_buy)
#                         print("States Sell: ", states_sell)
#                         print("Total Gains: ", total_gains)
#                         print("Investment: ", invest)

#                         # Update the logger with the simulation results
#                         logger.append(col + "," + str(force_buy) + "," + str(num_stocks) + "," + str(total_gains) + "," + str(invest))


In [121]:
import sys
# Define the max_days and conservative parameters
max_days = 3
conservative = True
initial_shares = 10

loggers = {}

for ticker in tt:
    loggers[ticker] = []

for ele in tt:
  initial = 10000/len(tt)         # Capital at starting point
  portfolio_cols = ['Bi-LSTM',	'CNN-Seq2Seq',	'Attention-CNN-LSTM',	'Dilated CNN Seq2Seq',	'CNN-LSTM',	'GRU-CNN',	'multimodal Predictions',	'LSTM Predictions',	'Polarity Predictions',	'Arima Predictions',		'GBTsPreds']
  # portfolio_cols = ['LSTM Predictions','Polarity Predictions','GBTsPreds','Arima Predictions','multimodal Predictions']
  models = ['Bi-LSTM',	'CNN-Seq2Seq',	'Attention-CNN-LSTM',	'Dilated CNN Seq2Seq',	'CNN-LSTM',	'GRU-CNN', 'multimodal', 'LSTM', 'Polarity', 'ARIMA', 'GBTs']

  buy_indicator = 0
  sell_indicator = 1

  # 'Ground Truth': incorrect metrics for models
  # max_stocks = [1, 5, 10, 20, 40, 80, 100]\
  max_stocks = [1, 5, 10]
  logger = []
  logger.append("Model,Indicators,Force Buy,Max Stocks,Gains USD,Gains Perc")

  # Load data: stocks and model predictions
  triker1 = ele.lower()
  df = pd.read_csv(data_port_path + triker1 + '_Simon/stocks_ts_'+ele+'_2021-1-4_2022-9-20.csv')
  df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
  filtered = df.loc[(df['date'] >= start_date)].loc[(df['date'] < end_date)]

  predictors = merge

  try:
      predictors['date'] = pd.to_datetime(predictors['Unnamed: 0'], format='%m/%d/%Y')
  except:
      predictors['dates'] = pd.to_datetime(predictors['Unnamed: 0'], format='%m/%d/%y')

  merged = predictors.merge(filtered, on='date')
  merged['NextDayTrend'] = merged['NextDayTrend'].replace({True: 1, False: 0})

  print(len(merged))
  merged

  # Simulation
  for col in portfolio_cols:
      # for force_buy in [True, False]:
      for force_buy in [True]:
          for num_stocks in max_stocks:
            for ticker, logger in loggers.items():
              if ele == ticker:
                print("\n\n\nTesting model " + col + " with force_buy=" + str(force_buy) + " and max_stocks=" + str(num_stocks) + ":\n")

                # Reset the initial state for each simulation scenario
                budget = initial
                shares = initial_shares

                # Call the buy_stock function to perform the simulation
                states_buy, states_sell, total_gains, invest = buy_stock(merged['close'], merged[col], initial_money=budget, max_buy=num_stocks, max_sell=num_stocks, force_buy=force_buy, conservative=True, max_days=3)

                # Print the results
                print("States Buy: ", states_buy)
                print("States Sell: ", states_sell)
                print("Total Gains: ", total_gains)
                print("Investment: ", invest)

                # Update the logger with the simulation results
                logger.append(col + "," + str(force_buy) + "," + str(num_stocks) + "," + str(total_gains) + "," + str(invest))
                # logger1.append(col + "," + str(force_buy) + "," + str(num_stocks) + "," + str(total_gains) + "," + str(invest))

# Buy Hold
  states_buy, states_sell, total_gains, invest = buy_hold_strategy(merged['close'], initial_money=initial)
  logger.append("Buy and hold" + ",NA,True," + str(total_gains) + "," + str(invest))

Streaming output truncated to the last 5000 lines.
Prediction: 1
day 67: cannot sell anything, inventory 0
Prediction: 1
day 68: cannot sell anything, inventory 0
Prediction: 1
day 69: cannot sell anything, inventory 0
Prediction: 1
day 70: cannot sell anything, inventory 0
Prediction: 1
day 71: cannot sell anything, inventory 0
Prediction: 1
day 72: cannot sell anything, inventory 0
Prediction: 1
day 73: cannot sell anything, inventory 0
Prediction: 1
day 74: cannot sell anything, inventory 0
Prediction: 1
day 75: cannot sell anything, inventory 0
Prediction: 1
day 76: cannot sell anything, inventory 0
Prediction: 1
day 77: cannot sell anything, inventory 0
Prediction: 1
day 78: cannot sell anything, inventory 0
Prediction: 1
day 79: cannot sell anything, inventory 0
Prediction: 1
day 80: cannot sell anything, inventory 0
Prediction: 1
day 81: cannot sell anything, inventory 0
Prediction: 1
day 82: cannot sell anything, inventory 0
Prediction: 1
day 83: cannot sell anything, inventory

In [122]:
# import sys

# loggers = {}

# for ticker in tt:
#     loggers[ticker] = []

# for ele in tt:
#   initial = 10000/len(tt)         # Capital at starting point
#   model_cols = ['LSTM Predictions','Polarity Predictions','GBTsPreds','Arima Predictions','multimodal Predictions']
#   models = ['LSTM','Polarity','GBTs','ARIMA', 'multimodal']

#   # 'Ground Truth': incorrect metrics for models
#   max_stocks = [1, 5, 10, 20, 40, 80, 100]
#   logger = []
#   logger.append("model,force_buy,max_stocks,gains_usd,gains_perc")

#   # LOAD DATA: STOCKS + MODEL PREDICTIONS
#   # Stock prices
#   triker1 = ele.lower()
#   df = pd.read_csv(data_port_path + triker1 + '_Simon/stocks_ts_'+ele+'_2021-1-4_2022-9-20.csv')
#   df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
#   filtered = df.loc[(df['date'] >= start_date)].loc[(df['date'] < end_date)]

#   # All base model predictors except stacking model: Note 2021 and 2022
#   predictors = merge

#   try:
#       predictors['date'] = pd.to_datetime(predictors['Unnamed: 0'], format='%m/%d/%Y')
#   except:
#       predictors['dates'] = pd.to_datetime(predictors['Unnamed: 0'], format='%m/%d/%y')

#   merged = predictors.merge(filtered, on='date')
#   # merged = merged.merge(stacking_preds, on='date')
#   merged['NextDayTrend'] = merged['NextDayTrend'].replace({True: 1, False: 0})

#   print(len(merged))
#   # merged.head()
#   merged
#   # Simulation with force_buy = True

  # for i in range(len(model_cols)):
  #     col = model_cols[i]
  #     model_name = models[i]

  #     for num_stocks in max_stocks:
  #       for ticker, logger in loggers.items():
  #         if ele == ticker:
  #           print("\n\n\nTesting model " + model_name + " with " + str(num_stocks) + " max_stocks: \n")
  #           states_buy, states_sell, total_gains, invest = buy_stock(merged['close'], merged[col], initial_money=initial, max_buy=num_stocks, max_sell=num_stocks, force_buy=True)
  #           logger.append(model_name + ",True," + str(num_stocks) + "," + str(total_gains) + "," + str(invest))
  #           print(ele + trend)
  #           plot_model_performance(ele + trend, merged['close'], states_buy, states_sell, model_name)

#generate

In [123]:
loggers.keys()

dict_keys(['ADBE', 'AMT', 'PLD', 'VICI', 'ATVI', 'CVS', 'BIO', 'JNJ', 'NVDA', 'IBM', 'NFLX', 'GOOG', 'AAPL', 'AMZN', 'EBAY', 'SBUX', 'TSLA'])

In [124]:
list(loggers)[0]

'ADBE'

In [125]:
for i in list(loggers):
  print(i)

ADBE
AMT
PLD
VICI
ATVI
CVS
BIO
JNJ
NVDA
IBM
NFLX
GOOG
AAPL
AMZN
EBAY
SBUX
TSLA


In [126]:
loggers

{'ADBE': ['Bi-LSTM,True,1,0.0,0.0',
  'Bi-LSTM,True,5,0.0,0.0',
  'Bi-LSTM,True,10,0.0,0.0',
  'CNN-Seq2Seq,True,1,0.0,0.0',
  'CNN-Seq2Seq,True,5,0.0,0.0',
  'CNN-Seq2Seq,True,10,0.0,0.0',
  'Attention-CNN-LSTM,True,1,0.0,0.0',
  'Attention-CNN-LSTM,True,5,0.0,0.0',
  'Attention-CNN-LSTM,True,10,0.0,0.0',
  'Dilated CNN Seq2Seq,True,1,0.0,0.0',
  'Dilated CNN Seq2Seq,True,5,0.0,0.0',
  'Dilated CNN Seq2Seq,True,10,0.0,0.0',
  'CNN-LSTM,True,1,0.0,0.0',
  'CNN-LSTM,True,5,0.0,0.0',
  'CNN-LSTM,True,10,0.0,0.0',
  'GRU-CNN,True,1,-1.89996337890625,-0.3229937744140625',
  'GRU-CNN,True,5,-1.89996337890625,-0.3229937744140625',
  'GRU-CNN,True,10,-1.89996337890625,-0.3229937744140625',
  'multimodal Predictions,True,1,0.0,0.0',
  'multimodal Predictions,True,5,0.0,0.0',
  'multimodal Predictions,True,10,0.0,0.0',
  'LSTM Predictions,True,1,-0.07000732421875,-0.0119012451171875',
  'LSTM Predictions,True,5,-0.07000732421875,-0.0119012451171875',
  'LSTM Predictions,True,10,-0.0700073242187

In [127]:
desired_lines = [line for line in loggers['TSLA'] if line.startswith('Buy and hold,NA,True,')]

for line in desired_lines:
    print(line)

Buy and hold,NA,True,-14.199951171875,-2.41399169921875
Buy and hold,NA,True,41.62005615234375,7.075409545898436
Buy and hold,NA,True,191.95999145507812,32.63319854736328
Buy and hold,NA,True,-20.339984893798828,-3.4577974319458007
Buy and hold,NA,True,-158.46002197265625,-26.938203735351564
Buy and hold,NA,True,150.2199935913086,25.537398910522462
Buy and hold,NA,True,0.0,0.0
Buy and hold,NA,True,19.049972534179688,3.2384953308105464
Buy and hold,NA,True,187.47998046875,31.871596679687496
Buy and hold,NA,True,-25.8896484375,-4.401240234375
Buy and hold,NA,True,78.550048828125,13.35350830078125
Buy and hold,NA,True,78.5360107421875,13.351121826171875
Buy and hold,NA,True,163.719970703125,27.832395019531248
Buy and hold,NA,True,-9.011993408203125,-1.5320388793945312
Buy and hold,NA,True,-24.1600341796875,-4.1072058105468745
Buy and hold,NA,True,14.149971008300668,2.405495071411113
Buy and hold,NA,True,261.61334228515625,44.474268188476564


In [128]:
#uptrend
#'Buy and hold,NA,True,959.1496887207031,172.64714477539062'
#downtrend
#'Buy and hold,NA,True,-1026.649962, -184.796209'

In [129]:
#100 thousand

#uptrend
#'Buy and hold,NA,True,19973.08 , 19.97'
#downtrend
#'Buy and hold,NA,True,-27889.51, -27.88'

In [130]:
#1 millions

#uptrend
#'Buy and hold,NA,True,128924.7645721439 , 232.06520341757814'
#downtrend
#'Buy and hold,NA,True,-226291.5075470524, -407.32470479042044'

In [131]:
aggregated_values = {}
for i in list(loggers):
  for logger in [loggers[list(loggers)[0]], loggers[list(loggers)[1]], loggers[list(loggers)[2]]
                 , loggers[list(loggers)[3]]
                 , loggers[list(loggers)[4]]
                 , loggers[list(loggers)[5]]
                 , loggers[list(loggers)[6]], loggers[list(loggers)[7]], loggers[list(loggers)[8]]
                 , loggers[list(loggers)[9]], loggers[list(loggers)[10]], loggers[list(loggers)[11]]
                 , loggers[list(loggers)[12]], loggers[list(loggers)[13]], loggers[list(loggers)[14]]
                 , loggers[list(loggers)[15]], loggers[list(loggers)[16]]
                #  , loggers[list(loggers)[17]]
                #  , loggers[list(loggers)[18]], loggers[list(loggers)[19]]
                #  , loggers[list(loggers)[20]]
                          ]:
      for entry in logger:
          model_name, force_buy, num_stocks, total_gains, invest = entry.split(",")

          key = (model_name, force_buy, num_stocks)
          if key not in aggregated_values:
              aggregated_values[key] = [float(total_gains), float(invest)]
          else:
              aggregated_values[key][0] += float(total_gains)
              aggregated_values[key][1] += float(invest)

# Format the output as a list
output_list = [",".join(list(key) + [str(val) for val in values]) for key, values in aggregated_values.items()]

output_list

['Bi-LSTM,True,1,13394.14260292053,2277.004242496492',
 'Bi-LSTM,True,5,1072.0338687896788,182.24575769424533',
 'Bi-LSTM,True,10,2573.1323966980044,437.43250743866116',
 'CNN-Seq2Seq,True,1,10232.494802474976,1739.524116420746',
 'CNN-Seq2Seq,True,5,2821.771858215332,479.7012158966069',
 'CNN-Seq2Seq,True,10,5442.828815460205,925.2808986282354',
 'Attention-CNN-LSTM,True,1,11307.931018829346,1922.348273200988',
 'Attention-CNN-LSTM,True,5,296.81071090699015,50.45782085418832',
 'Attention-CNN-LSTM,True,10,2288.0188484191963,388.96320423126355',
 'Dilated CNN Seq2Seq,True,1,10569.525619506836,1796.8193553161618',
 'Dilated CNN Seq2Seq,True,5,5427.703495025633,922.7095941543585',
 'Dilated CNN Seq2Seq,True,10,7021.113033294678,1193.5892156600944',
 'CNN-LSTM,True,1,14484.254795074463,2462.3233151626578',
 'CNN-LSTM,True,5,5501.056785583497,935.1796535491934',
 'CNN-LSTM,True,10,4369.537021636968,742.8212936782844',
 'GRU-CNN,True,1,7853.452569961549,1335.0869368934636',
 'GRU-CNN,True,5

In [132]:
ns = str(len(tt))
# np.savetxt(gcn_file_path + 'GCN_'+folder + sc + ns +'_stock' + trend + "_portfolio_summary_100thousands.csv", output_list, delimiter=" ", newline = "\n", fmt="%s")
np.savetxt('/content/drive/MyDrive/stock_GCN_info/multiProt/' + sc + ns +'_stock' + trend + "_portfolio_summary_10thousands.csv", output_list, delimiter=" ", newline = "\n", fmt="%s")